In [38]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [39]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Travel Agent")

LangSmith 추적을 시작합니다.
[프로젝트명]
Travel Agent


In [40]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# CSV 로더 생성
loader = CSVLoader(file_path="./data/부산명소 국문 정보.csv")

# 데이터 로드
docs = loader.load()

print(len(docs))
print(docs[0])

154
page_content='﻿콘텐츠ID: 255
콘텐츠명: 흰여울문화마을
구군: 영도구
위도: 35.07885
경도: 129.04402
여행지: 흰여울문화마을
제목: 가파른 절벽 끝에 흰여울문화마을
부제목: 흰여울길에서 만난 느림의 미학
주소: 부산광역시 영도구 흰여울길
연락처: 051-419-4067
홈페이지: http://www.ydculture.com/huinnyeoulculturetown/
교통정보: 도시철도 1호선 남포역 6번 출구 → 7, 71, 508 버스 환승 → 영선동 백련사 하차
버스 7,71,508 영선동 백련사 하차
주차 절영해안산책로 입구 공영 주차장 (유료)
운영일: 
휴무일: 
운영 및 시간: 매일
이용요금: 무료
편의시설: 장애인 주차장(절영해안산책로 공영주차장, 유료)
이미지URL: https://www.visitbusan.net/uploadImgs/files/cntnts/20191222164810529_ttiel
썸네일이미지URL: https://www.visitbusan.net/uploadImgs/files/cntnts/20191222164810529_thumbL
상세내용: 절영해안산책로 가파른 담벼락 위로 독특한 마을 풍경이 보인다. 해안가 절벽 끝에 바다를 따라 난 좁은 골목길 안쪽으로 작은 집들이 다닥다닥 붙어 있다. 피난민들의 애잔한 삶이 시작된 곳이자 현재는 마을주민과 함께하는 문화마을공동체 흰여울문화마을이 자리한 곳이다.
부산의 대표적인 원도심 흰여울길은 봉래산 기슭에서 굽이쳐 내리는 물줄기가 마치 흰 눈이 내리는 모습과 비슷하다 하여 이름 지어졌다. 2011년 12월, 낡은 가옥을 리모델링하면서 현재는 영도의 생활을 느낄 수 있는 독창적인 문화예술마을로 거듭났다. 

영화 ＜변호인＞을 보지 못한 사람도 그 영화의 명대사는 외울 것이다. 진우네 집 담벼락에 크게 쓰여 있기 때문이다. 많은 여행자의 영도 여행 인증샷 1호는 언제나 진우네 담벼락이었다. 흰여울길은 모두 14개의 골목길로 이어져 있고 골목골목 아기자

In [42]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

def load_and_split_csv(file_path, chunk_size=500, chunk_overlap=50):
    # CSV 로더 생성 및 데이터 로드
    loader = CSVLoader(file_path=file_path)
    docs = loader.load()
    
    # 텍스트 분할기 생성
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    
    split_docs = []
    for doc in docs:
        content = doc.page_content
        metadata = doc.metadata
        
        # '상세내용:' 이전과 이후의 텍스트 분리
        parts = content.split('상세내용:', 1)
        main_content = parts[0].strip()
        detailed_content = parts[1].strip() if len(parts) > 1 else ""
        
        if detailed_content:
            splits = text_splitter.split_text(detailed_content)
            for i, split in enumerate(splits):
                # 모든 필드를 포함하면서 상세내용만 분할
                split_content = f"{main_content}\n상세내용: {split}"
                split_docs.append(Document(
                    page_content=split_content,
                    metadata={**metadata, 'split_id': i, 'total_splits': len(splits)}
                ))
        else:
            # 상세내용이 없으면 원본 문서 그대로 추가
            split_docs.append(doc)
    
    return split_docs

# 사용 예시
file_path = "./data/부산명소 국문 정보.csv"

split_docs = load_and_split_csv(file_path)

print(f"원본 문서 수: {len(CSVLoader(file_path=file_path).load())}")
print(f"분할된 문서 수: {len(split_docs)}")
print(f"\n첫 번째 분할 문서의 내용:\n{split_docs[3].page_content}")
print(f"\n첫 번째 분할 문서의 메타데이터:\n{split_docs[3].metadata}")

원본 문서 수: 154
분할된 문서 수: 474

첫 번째 분할 문서의 내용:
﻿콘텐츠ID: 256
콘텐츠명: 깡깡이 예술마을
구군: 영도구
위도: 35.092648
경도: 129.03255
여행지: 깡깡이 예술마을
제목: 역사가 살아 숨 쉬는 깡깡이예술마을
부제목: 역사와 예술이 공존하는 곳
주소: 부산광역시 영도구 대평북로 36 깡깡이 안내센터
연락처: 051-418-3336
홈페이지: http://kangkangee.com
교통정보: 도시철도 1호선 남포역 (6번출구) 도보 18분 
마을버스 영도구2 대평동 하차 
주차 봉래시장, 남항시장 공영주차장 (유료)
운영일: 
휴무일: 매주 월요일 (투어 등 시설)
운영 및 시간: 평일, 주말 10:00~17:00
가급적 평일 낮 시간 제외, 주말 혹은 통합투어 프로그램 이용 요망
이용요금: 프로그램별 상이
편의시설: 휠체어 접근 가능
이미지URL: https://www.visitbusan.net/uploadImgs/files/cntnts/20191222171209005_ttiel
썸네일이미지URL: https://www.visitbusan.net/uploadImgs/files/cntnts/20191222171209005_thumbL
상세내용: 영도다리 건너 자갈치시장 맞은 편 물양장에 배들이 가득 들어차 있다. 세상에 못 고치는 배는 없다는 전설적인 수리조선의 메카 깡깡이 예술마을이 있는 곳이다. 깡깡이란 말은 수리 조선소에서 배 표면에 녹이 슬어 너덜해진 페인트나 조개껍데기를 망치로 두드려 벗겨낼 때 깡깡 소리가 난다 하여 생겨난 말이다. 마을 전체에 깡깡 소리가 그칠 날이 없었던 그 시절부터 대평동은 그렇게 깡깡이 예술마을로 불리게 되었다.
이런 깡깡이 예술마을의 유래 때문인지 깡깡 소리를 내는 일꾼이 남성 기술자인 줄 알지만 전혀 아니다. 밧줄 하나에 몸을 의지한 채 오랜 세월 그 힘든 깡깡이질을 해낸 사람은 다름 아닌 대평동 깡깡이 아지매들이다. 자식에게만큼은 가난을 물려주지 않기 위해 난청과 이명을 이

In [43]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_docs, embedding=OpenAIEmbeddings())

# 단계 5: Dense Retriever 생성
# 벡터DB에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [48]:
from langchain_core.prompts import PromptTemplate

travel_planner_template = """
당신은 여행 일정 생성 AI입니다. 다음 지침에 따라 작업을 수행하세요:

시스템 인스트럭션: 여행 일정 생성 AI

1. 사용자 정보 수집:
사용자로부터 여행 취향과 관련된 정보를 수집합니다. 예를 들어, 선호하는 여행 테마, 활동, 음식, 이동 수단 등을 확인합니다.

2. 데이터 세트 필터링:
수집된 사용자 정보를 바탕으로, 제공된 부산 관련 데이터 세트(예: 관광 명소, 맛집, 테마 여행 정보 등)를 필터링하여 사용자의 여행 성향에 맞는 장소와 활동을 선택합니다.

3. 맞춤형 여행 일정 생성:
필터링된 데이터를 바탕으로, 사용자의 여행 기간과 일정에 맞는 구체적이고 체계적인 일정을 생성합니다.
일정에는 추천 방문지, 식사 장소, 이동 경로 및 시간 배분 등이 포함되어야 합니다.
또한, 일정 별 근처에 있는 지하철 역 정보를 상황에 맞게 제공해야 합니다.

4. 사용자 피드백 반영:
생성된 일정에 대해 사용자가 피드백을 제공할 경우, 일정을 수정하여 더욱 맞춤화된 계획을 제공합니다.

5. 일정 제공 및 안내:
최종 확정된 여행 일정을 사용자에게 제공하고, 일정의 각 단계에 대한 간단한 설명과 이유를 함께 제시합니다.
또한, 가게나 명소에 대한 정보를 자세히 제공해야 합니다.

다음은 부산의 음식에 대한 정보입니다:
{sights_info}

이 정보를 바탕으로 체계적인 여행 일정을 작성해서 답해 주세요.

다음은 예시입니다:

예시 1:
사용자: 2024년 8월 13일부터 1박 2일 간 부산 여행을 가려고 해. 나는 해산물 음식을 좋아하고, 유명한 식당 위주로 여행을 다니고 싶어.

### 부산 1박 2일 유명 식당 중심 여행 일정 안내

#### 첫째 날 (2024년 8월 13일)
- 11:00: 부산역 도착
- 12:00: 점심 - 부산역 근처 유명한 식당 '만드리곤드레밥'
- 13:30: 부산역에서 택시를 이용하여 해운대로 이동
- 14:00: 관광 - 해운대 해수욕장
- 18:00: 저녁 - 해운대 유명한 식당 '사미헌'
- 20:00: 숙소 체크인 및 휴식

#### 둘째 날 (2024년 8월 14일)
- 09:00: 아침 - 호텔 조식 또는 근처 빵집에서 아침 식사
- 10:00: 점심 - 부산 중심지 근처 유명한 식당 '부흥식당'
- 12:00: 가볍게 산책이나 쇼핑을 즐기며 시간 보내기
- 15:00: 기념품 구매를 위해 부산 중심지로 이동
- 18:00: 저녁 - 부산 중심지 유명한 식당 '부다면옥'
- 19:30: 부산역으로 이동
- 20:00: 기차 승차

#### 유명 식당 및 관광지
- 첫째 날 점심: 만드리곤드레밥 (부산역 근처)
- 첫째 날 저녁: 사미헌 (해운대)
- 둘째 날 점심: 부흥식당 (부산 중심지)
- 둘째 날 저녁: 부다면옥 (부산 중심지)

#### 이동 경로
- 부산역 -> 만드리곤드레밥: 택시 또는 대중교통 이용
- 만드리곤드레밥 -> 해운대: 택시 이용
- 해운대 -> 사미헌: 택시 이용
- 사미헌 -> 숙소: 택시 이용
- 숙소 -> 부흥식당: 택시 또는 대중교통 이용
- 부흥식당 -> 부다면옥: 택시 이용
- 부다면옥 -> 부산역: 택시 또는 대중교통 이용

부산 여행을 즐기시고 즐거운 시간 보내시길 바랍니다! 여행 일정이나 추가 정보가 필요하시면 언제든지 물어보세요.

이제 사용자의 요청을 처리해 주세요:

사용자 요청: {user_request}

단계별로 생각해봅시다:
1. 사용자의 여행 기간과 선호도를 파악합니다.
2. 사용자가 원하는 특별한 장소나 음식이 있는지 확인합니다.
3. 사용자가 방문을 원하지 않는 장소가 있는지 확인합니다.
4. 제공된 정보를 바탕으로 적절한 식당과 관광지를 선택합니다.
5. 선택한 장소들을 고려하여 효율적인 동선을 계획합니다.
6. 각 날짜와 시간대별로 상세한 일정을 작성합니다.
7. 이동 방법과 주변 지하철역 정보를 포함시킵니다.
8. 선택한 식당과 관광지에 대한 간단한 설명을 추가합니다.

이제 위의 단계를 따라 사용자를 위한 맞춤형 여행 계획을 작성해 주세요.

### 부산 여행 일정 안내

[여기에 일정을 작성해 주세요]

"""

prompt = PromptTemplate.from_template(travel_planner_template)

In [49]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# RAG 체인 생성
rag_chain = (
    {
        "sights_info": retriever,
        "user_request": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [50]:
from langchain_teddynote.messages import stream_response

user_request = """
사용자는 2024년 8월 15일부터 1박 2일 동안 부산 여행을 계획하고 있습니다. 
그는 해산물 음식을 싫어하며, 유명한 식당과 명소를 중심으로 여행을 하고 싶어합니다.

우선, 여행 기간을 확인합니다: 1박 2일, 즉 2일 동안의 일정입니다. 
사용자는 유명한 식당과 명소를 위주로 방문하기를 원합니다.
또한, 위치 간 동선이 체계적으로 고려되어야 합니다. 
주요 목표는 *테마가 있는 여행*을 하는 것입니다.

하지만, 사용자는 최근 여행으로 영도에 방문한 경험이 있어 영도가 일정에 포함되는 것을 원치 않습니다.
사용자는 8월 15일 11시에 부산역에 도착하고, 다음 날 저녁 20시에 부산역에서 기차를 타고 서울로 돌아갑니다.
또한, 사용자는 부산에만 있는 이재모 피자를 먹고 싶어합니다. 가게 정보를 자세히 제공해주세요.

첫째 날, 사용자는 기차를 타고 부산역에 도착합니다. 
도착 후, 부산의 유명한 식당 중 점심과 저녁 식사 장소를 선정합니다.
점심 식사 후에는 근처의 유명한 관광지를 방문할 수 있도록 계획을 세웁니다.
저녁에는 두 번째 유명한 식당을 예약하고 방문합니다.

둘째 날, 아침 식사는 호텔 또는 근처의 간단한 식당에서 해결합니다.
호텔의 위치는 서면입니다.
점심 식사는 또 다른 유명한 식당에서 예약합니다. 
오후에는 가볍게 산책하거나 쇼핑을 하며 시간을 보냅니다.
저녁에는 부산의 대표적인 야경을 감상합니다.
각 단계에서 방문할 식당과 장소를 구체적으로 선정하고, 이동 경로와 시간을 배분합니다.
"""

answer = rag_chain.stream(user_request)
stream_response(answer)

### 부산 1박 2일 유명 식당 및 명소 중심 여행 일정 안내

#### 첫째 날 (2024년 8월 15일)
- **11:00**: 부산역 도착
- **11:30**: 점심 - 이재모 피자 (부산역 근처)
  - **주소**: 부산광역시 동구 중앙대로 214번길 7
  - **설명**: 부산에서만 맛볼 수 있는 이재모 피자는 얇고 바삭한 도우와 신선한 재료로 유명합니다.
- **13:00**: 관광 - 부산 트래블라운지
  - **주소**: 부산광역시 동구 대영로243번길 38 1, 2층
  - **설명**: 부산의 다양한 체험 프로그램을 제공하는 곳으로, SNS 보물찾기 미션과 야경 촬영 노하우를 배울 수 있습니다.
  - **이동**: 도보 (부산역 1번 출구에서 도보 2분)
- **15:00**: 관광 - 용두산공원
  - **주소**: 부산광역시 중구 용두산길 37-55
  - **설명**: 부산의 대표적인 공원으로, 부산타워에서 부산 전경을 감상할 수 있습니다.
  - **이동**: 택시 (약 10분 소요)
- **17:00**: 관광 - 국제시장
  - **주소**: 부산광역시 중구 신창동4가 21
  - **설명**: 다양한 먹거리와 쇼핑을 즐길 수 있는 부산의 대표적인 전통시장입니다.
  - **이동**: 도보 (용두산공원에서 도보 10분)
- **19:00**: 저녁 - 할매가야밀면
  - **주소**: 부산광역시 중구 광복로 56-14
  - **설명**: 부산의 대표적인 밀면 맛집으로, 시원하고 쫄깃한 밀면을 맛볼 수 있습니다.
  - **이동**: 도보 (국제시장에서 도보 10분)
- **21:00**: 숙소 체크인 및 휴식 (서면)

#### 둘째 날 (2024년 8월 16일)
- **09:00**: 아침 - 호텔 조식 또는 근처 카페에서 간단한 아침 식사
- **10:00**: 관광 - 서면 쇼핑 거리
  - **주소**: 부산광역시 부산진구 서면로 68
  - **설명**: 다양한 쇼핑과 먹거리를 즐길 수 있는 부산의 대표적인 번화